<a href="https://colab.research.google.com/github/pawel0508/NeuralNetworks/blob/main/overfitting_underfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Główne problemy uczenia maszynowego: overfitting and underfitting**.

In [35]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle 

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets.imdb import get_word_index
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sns.set()
tf.__version__

'2.3.0'

### **Załadowanie i przygotowanie danych**.


In [36]:
NUM_WORDS = 10000
INDEX_FROM = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = NUM_WORDS, index_from = INDEX_FROM)

print(f'train data shape: {train_data.shape}')
print(f'train labels: {train_labels.shape}')
print(f'test data shape: {test_data.shape}')
print(f'test labels: {test_labels.shape}')

train data shape: (25000,)
train labels: (25000,)
test data shape: (25000,)
test labels: (25000,)


In [37]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [38]:
word_to_idx = get_word_index()

In [39]:
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<START>'] = 1
word_to_idx['<UNK>'] = 2
word_to_idx['<UNUSED>'] = 3
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [41]:
def multi_hot_sequences(sequences, dimension):
  results = np.zeros((len(sequences), dimension))
  for i, word_indices in enumerate(sequences):
    results[i, word_indices] = 1
  return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
train_data.shape


IndexError: ignored

In [ ]:
test_data.shape

### Budowa modelu

In [42]:
baseline_model = Sequential()
baseline_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
baseline_model.add(Dense(16, activation='relu'))
baseline_model.add(Dense(1, activation='sigmoid'))

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 16)                160016    
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [43]:
baseline_history = baseline_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 2s 50ms/step - loss: 0.4820 - accuracy: 0.8072 - binary_crossentropy: 0.4820 - val_loss: 0.3387 - val_accuracy: 0.8772 - val_binary_crossentropy: 0.3387
Epoch 2/20
49/49 [==============================] - 2s 36ms/step - loss: 0.2496 - accuracy: 0.9114 - binary_crossentropy: 0.2496 - val_loss: 0.2836 - val_accuracy: 0.8878 - val_binary_crossentropy: 0.2836
Epoch 3/20
49/49 [==============================] - 2s 36ms/step - loss: 0.1829 - accuracy: 0.9361 - binary_crossentropy: 0.1829 - val_loss: 0.2892 - val_accuracy: 0.8839 - val_binary_crossentropy: 0.2892
Epoch 4/20
49/49 [==============================] - 2s 37ms/step - loss: 0.1451 - accuracy: 0.9497 - binary_crossentropy: 0.1451 - val_loss: 0.3098 - val_accuracy: 0.8805 - val_binary_crossentropy: 0.3098
Epoch 5/20
49/49 [==============================] - 2s 38ms/step - loss: 0.1178 - accuracy: 0.9621 - binary_crossentropy: 0.1178 - val_loss: 0.3314 - val_accuracy: 0.8767 - val_bin

In [23]:
pd.DataFrame(baseline_history.history)

,loss,accuracy,binary_crossentropy,val_loss,val_accuracy,val_binary_crossentropy
0,0.490113,0.80560,0.490113,0.333117,0.87852,0.333117
1,0.246509,0.91140,0.246509,0.281680,0.88868,0.281680
2,0.182804,0.93588,0.182804,0.291961,0.88292,0.291961
3,0.145983,0.95132,0.145983,0.309266,0.87904,0.309266
4,0.121575,0.96052,0.121575,0.333947,0.87580,0.333947
5,0.102413,0.96748,0.102413,0.365916,0.87068,0.365916
6,0.086968,0.97400,0.086968,0.401751,0.86656,0.401751
7,0.073640,0.98000,0.073640,0.441723,0.86216,0.441723
8,0.062793,0.98444,0.062793,0.480215,0.86048,0.480215
9,0.052844,0.98716,0.052844,0.524418,0.85600,0.524418


Mniejszy model

In [24]:
smaller_model = Sequential()
smaller_model.add(Dense(4, activation='relu', input_shape=(NUM_WORDS,)))
smaller_model.add(Dense(4, activation='relu'))
smaller_model.add(Dense(1, activation='sigmoid'))

smaller_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

smaller_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 4)                 40004     
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 40,029
Trainable params: 40,029
Non-trainable params: 0
_________________________________________________________________


In [25]:
smaller_history = smaller_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 2s 49ms/step - loss: 0.6220 - accuracy: 0.6835 - binary_crossentropy: 0.6220 - val_loss: 0.5533 - val_accuracy: 0.8082 - val_binary_crossentropy: 0.5533
Epoch 2/20
49/49 [==============================] - 2s 34ms/step - loss: 0.4718 - accuracy: 0.8612 - binary_crossentropy: 0.4718 - val_loss: 0.4309 - val_accuracy: 0.8676 - val_binary_crossentropy: 0.4309
Epoch 3/20
49/49 [==============================] - 2s 33ms/step - loss: 0.3524 - accuracy: 0.8999 - binary_crossentropy: 0.3524 - val_loss: 0.3514 - val_accuracy: 0.8799 - val_binary_crossentropy: 0.3514
Epoch 4/20
49/49 [==============================] - 2s 32ms/step - loss: 0.2783 - accuracy: 0.9158 - binary_crossentropy: 0.2783 - val_loss: 0.3129 - val_accuracy: 0.8855 - val_binary_crossentropy: 0.3129
Epoch 5/20
49/49 [==============================] - 2s 33ms/step - loss: 0.2336 - accuracy: 0.9264 - binary_crossentropy: 0.2336 - val_loss: 0.2928 - val_accuracy: 0.8880 - val_bin

Wiekszy model

In [26]:
bigger_model = Sequential()
bigger_model.add(Dense(512, activation='relu', input_shape=(NUM_WORDS,)))
bigger_model.add(Dense(512, activation='relu'))
bigger_model.add(Dense(1, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

bigger_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               5120512   
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 513       
Total params: 5,383,681
Trainable params: 5,383,681
Non-trainable params: 0
_________________________________________________________________


In [27]:
bigger_history = bigger_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 15s 316ms/step - loss: 0.3439 - accuracy: 0.8541 - binary_crossentropy: 0.3439 - val_loss: 0.3063 - val_accuracy: 0.8740 - val_binary_crossentropy: 0.3063
Epoch 2/20
49/49 [==============================] - 14s 291ms/step - loss: 0.1486 - accuracy: 0.9453 - binary_crossentropy: 0.1486 - val_loss: 0.3278 - val_accuracy: 0.8742 - val_binary_crossentropy: 0.3278
Epoch 3/20
49/49 [==============================] - 14s 288ms/step - loss: 0.0564 - accuracy: 0.9822 - binary_crossentropy: 0.0564 - val_loss: 0.4277 - val_accuracy: 0.8690 - val_binary_crossentropy: 0.4277
Epoch 4/20
49/49 [==============================] - 14s 288ms/step - loss: 0.0094 - accuracy: 0.9985 - binary_crossentropy: 0.0094 - val_loss: 0.5569 - val_accuracy: 0.8683 - val_binary_crossentropy: 0.5569
Epoch 5/20
49/49 [==============================] - 14s 287ms/step - loss: 0.0019 - accuracy: 1.0000 - binary_crossentropy: 0.0019 - val_loss: 0.6523 - val_accuracy: 0.8701

In [31]:
hist = pd.DataFrame(baseline_history.history)
hist['epoch'] = baseline_history.epoch
hist.head()

,loss,accuracy,binary_crossentropy,val_loss,val_accuracy,val_binary_crossentropy,epoch
0,0.490113,0.80560,0.490113,0.333117,0.87852,0.333117,0
1,0.246509,0.91140,0.246509,0.281680,0.88868,0.281680,1
2,0.182804,0.93588,0.182804,0.291961,0.88292,0.291961,2
3,0.145983,0.95132,0.145983,0.309266,0.87904,0.309266,3
4,0.121575,0.96052,0.121575,0.333947,0.87580,0.333947,4


In [33]:

import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['smaller', 'baseline', 'bigger'], [smaller_history, baseline_history, bigger_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy', template = 'simple_white')
fig.show()

In [48]:
from tensorflow.keras.regularizers import l2
l2_model = Sequential()
l2_model.add(Dense(units = 16, kernel_regularizer = l2(0.001), activation = 'relu', input_shape = (NUM_WORDS,)))
l2_model.add(Dense(units = 16, kernel_regularizer = l2(0.01), activation = 'relu'))
l2_model.add(Dense(1, activation='sigmoid'))


l2_model.compile(optimizer = 'adam',
                 loss = 'binary_crossentropy',
                 metrics = ['accuracy', 'binary_crossentropy'])


In [49]:
l2_model_history = l2_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 2s 48ms/step - loss: 0.6285 - accuracy: 0.8156 - binary_crossentropy: 0.4633 - val_loss: 0.4880 - val_accuracy: 0.8760 - val_binary_crossentropy: 0.3319
Epoch 2/20
49/49 [==============================] - 2s 37ms/step - loss: 0.4035 - accuracy: 0.9120 - binary_crossentropy: 0.2520 - val_loss: 0.4323 - val_accuracy: 0.8876 - val_binary_crossentropy: 0.2878
Epoch 3/20
49/49 [==============================] - 2s 38ms/step - loss: 0.3439 - accuracy: 0.9287 - binary_crossentropy: 0.2048 - val_loss: 0.4193 - val_accuracy: 0.8843 - val_binary_crossentropy: 0.2861
Epoch 4/20
49/49 [==============================] - 2s 38ms/step - loss: 0.3103 - accuracy: 0.9385 - binary_crossentropy: 0.1811 - val_loss: 0.4124 - val_accuracy: 0.8844 - val_binary_crossentropy: 0.2878
Epoch 5/20
49/49 [==============================] - 2s 38ms/step - loss: 0.2880 - accuracy: 0.9455 - binary_crossentropy: 0.1666 - val_loss: 0.4219 - val_accuracy: 0.8771 - val_bin

In [50]:
fig = go.Figure()
for name, history in zip(['baseline', 'l2'], [baseline_history, l2_model_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()


In [51]:
from tensorflow.keras.layers import Dropout

dropout_model = Sequential()
dropout_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(16, activation='relu'))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(1, activation='sigmoid'))

dropout_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dropout_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 16)                160016    
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 16)                272       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [52]:
dropout_history = dropout_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 8s 160ms/step - loss: 0.6150 - accuracy: 0.6565 - binary_crossentropy: 0.6150 - val_loss: 0.4623 - val_accuracy: 0.8549 - val_binary_crossentropy: 0.4623
Epoch 2/20
49/49 [==============================] - 2s 36ms/step - loss: 0.4436 - accuracy: 0.8130 - binary_crossentropy: 0.4436 - val_loss: 0.3343 - val_accuracy: 0.8807 - val_binary_crossentropy: 0.3343
Epoch 3/20
49/49 [==============================] - 2s 34ms/step - loss: 0.3418 - accuracy: 0.8737 - binary_crossentropy: 0.3418 - val_loss: 0.2860 - val_accuracy: 0.8874 - val_binary_crossentropy: 0.2860
Epoch 4/20
49/49 [==============================] - 2s 35ms/step - loss: 0.2804 - accuracy: 0.9030 - binary_crossentropy: 0.2804 - val_loss: 0.2752 - val_accuracy: 0.8872 - val_binary_crossentropy: 0.2752
Epoch 5/20
49/49 [==============================] - 2s 35ms/step - loss: 0.2424 - accuracy: 0.9196 - binary_crossentropy: 0.2424 - val_loss: 0.2790 - val_accuracy: 0.8873 - val_bi

In [53]:
fig = go.Figure()
for name, history in zip(['baseline', 'dropout'], [baseline_history, dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()